In [1]:
import deepquantum as dq
import numpy as np
import torch
print('version',dq.__version__)

version 3.3.1


# 基于Fock 后端构建光量子线路

## Fock态

光量子线路Fock态（量子数态）可以表示为``|n_1,n_2,...,n_m>``，其中 ``n_k`` 表示第 ``k`` 模的光子数。在DeepQuantum photonic模块中，Fock态用 ``FockState`` 表示。

我们利用 ``FockState`` 准备一个3模的量子态，其中第一、三模光子数为1， 第二模为0：


In [2]:
qstate = dq.FockState(state=[1,0,1])
print(qstate)

|101>


其数据为torch的tensor，存在属性 ``state`` 中：

In [3]:
print(qstate.state)

tensor([1, 0, 1])


输入态的类型也可以是 ``torch.tensor`` ，并支持 ``batch`` 输入：

In [4]:
tc_state = torch.tensor([1,0,1])
qstate = dq.FockState(state=tc_state)
print(qstate)

batch_state = torch.randint(0, 2, (4, 3))
print(batch_state)
qstate = dq.FockState(state=batch_state)
print(qstate)

|101>
tensor([[1, 1, 1],
        [1, 0, 0],
        [1, 1, 1],
        [0, 1, 1]])
state_0: |111>
state_1: |100>
state_2: |111>
state_3: |011>



``state`` 的输入也可以是真空态 `` `vac` ``或全零态`` `zeros` ``， 表示所有模式均无光子：

In [5]:
vaccum_state = dq.FockState(state='vac',nmode=3)
print(vaccum_state)

zeros_state = dq.FockState(state='zeros',nmode=3)
print(zeros_state)

|000>
|000>


其中，可选输入参数 ``nmode`` 表示模式数。当模式数小于 ``state`` 代表的模式数时， 会根据 ``nmode`` 截断：

In [6]:
tc_state = torch.tensor([1, 0, 1, 0])
qstate = dq.FockState(state=tc_state, nmode=3)
print(qstate)

|101>


当量子态空间较大时，可以设定光子截断数 ``cutoff`` 来降低计算复杂度，表示每模的光子数上限+1。默认值为单模最大光子数+1。

In [7]:
tc_state = torch.tensor([8, 5, 7])
qstate = dq.FockState(state=tc_state, cutoff=9)
print(qstate)

|857>


In [8]:
cir = dq.QumodeCircuit(nmode=3,init_state =[2,2,2],cutoff=3)
cir()
cir.measure(with_prob=True)

{|222>: (1024, tensor([1.]))}

最后一个可选参数 ``basis`` 默认为 ``True``, 当输入为叠加态（如 $\frac{\sqrt{2}}{2}$[1,0,1,0],$\frac{\sqrt2}{2}$[0,1,1,0]）时，需要设置为 ``False``。此时fock态由带batch的张量积表示，cutoff为总光子数和+1：

In [9]:
qstate = dq.FockState(state = [(np.sqrt(2)/2, [1, 0, 1, 0]), (np.sqrt(2)/2, [0, 1, 1, 0])], basis = False)
print(qstate)

#得到batch形式的fock state tensor
print(qstate.state.size())

state_0: (0.707+0.000j)|0110> + (0.707+0.000j)|1010>

torch.Size([1, 3, 3, 3, 3])


另外， ``FockState`` 支持根据字典序比较大小，从而实现对采样后的字典根据fock态进行排序：

In [10]:
state_a = dq.FockState(state = [0,0,1])
state_a2 = dq.FockState(state = [0,0,1])
state_b = dq.FockState(state = [0,0,2])

print(state_a == state_a2)
print(state_a < state_b)

# 构建一个简单Qunode线路
cir = dq.QumodeCircuit(nmode = 3, init_state = [1,2,0])
cir.bs(wires=[0,1])
cir.bs(wires=[1,2])
#前向函数
cir()
# 采样测量
result_dict = cir.measure()
print('采样结果（默认按样本数由高到低排序）：', result_dict)

# 对采样结果按fock态排序
sorted_dict = dict(sorted(result_dict.items(), key=lambda x: x[0], reverse=True))
print('采样结果（按样本态字典序排序）：', sorted_dict)

True
True
采样结果（默认按样本数由高到低排序）： {|210>: 385, |300>: 266, |120>: 277, |030>: 34, |111>: 28, |201>: 24, |021>: 7, |102>: 3}
采样结果（按样本态字典序排序）： {|300>: 266, |210>: 385, |201>: 24, |120>: 277, |111>: 28, |102>: 3, |030>: 34, |021>: 7}


## 构建光量子线路

### 初始化

光量子线路对应着DeepQuantum photonic ``QumodeCircuit`` 类。仅需指定空间模式数 ``nmode``和初态 ``init_state``，即可完成初始化：

In [11]:
cir = dq.QumodeCircuit(nmode=3, init_state=[1,0,1])

类似 ``FockState`` ，``init_state`` 也支持torch.tensor输入，以及 ``FockState`` 输入：

In [12]:
cir1 = dq.QumodeCircuit(nmode=3, init_state=torch.tensor([1,0,1]))

fockstate = dq.FockState(state=[1, 0, 1])
cir2 = dq.QumodeCircuit(nmode=3, init_state=fockstate)

### 量子门

初始化后，可以加入各种光量子门

当前模拟器支持加入下列门：
1. ps 门： ps参数表示相移器对应的相位角 $e^{i\theta}$  
2. bs门： bs有两个参数，对应的U矩阵如下
$$
\begin{pmatrix}
\cos{\theta}&-e^{-i\phi}\sin{\theta}\\
e^{i\phi}sin{\theta}&\cos{\theta}\\
\end{pmatrix}
$$

但是bs门一般默认只有一个参数, $bs_\theta$和$bs_\phi$.\
对于$bs_\theta$($\phi = \pi/2$)， 当$\theta$为$\pi/4$ 时，对应的BS门为 
$$\frac{\sqrt{2}}{2}
\begin{pmatrix}
1&i\\
i&1\\
\end{pmatrix}
$$
对应的分束比为1:1 \
当$\sin{\theta} = \sqrt{1/3}$, $\theta=0.61547$ 时， 对应的分束比为1：2, 对应的BS门为:

$$\sqrt{\frac{1}{3}}
\begin{pmatrix}
\sqrt{2}&i\\
i&\sqrt{2}\\
\end{pmatrix}
$$
3. H 门 
$$
\begin{pmatrix}
\cos{\frac{\theta}{2}}&\sin{\frac{\theta}{2}}\\
\sin{\frac{\theta}{2}}&-\cos{\frac{\theta}{2}}\\
\end{pmatrix}
$$
4. Rx 门 
$$
\begin{pmatrix}
\cos{\frac{\theta}{2}}&i\sin{\frac{\theta}{2}}\\
i\sin{\frac{\theta}{2}}&\cos{\frac{\theta}{2}}\\
\end{pmatrix}
$$
5. Ry 门
$$
\begin{pmatrix}
\cos{\frac{\theta}{2}}&-\sin{\frac{\theta}{2}}\\
\sin{\frac{\theta}{2}}&\cos{\frac{\theta}{2}}\\
\end{pmatrix}
$$
6. 50：50 分束器 (directional coupler)
$$\frac{\sqrt{2}}{2}
\begin{pmatrix}
1&i\\
i&1\\
\end{pmatrix}
$$
7. mzi 包括两种， 一种是先 $\phi$后 $\theta$，一种是先 $\theta$后 $\phi$, 对应的矩阵如下
$$
MZI_{PT} = 
ie^{i\theta/2}
\begin{pmatrix}
e^{i\phi}\sin{\theta}&\cos{\theta}\\
e^{i\phi}\cos{\theta}&-\sin{\theta}\\
\end{pmatrix}
$$

$$
MZI_{TP} = 
ie^{i\theta/2}
\begin{pmatrix}
e^{i\phi}\sin{\theta}&e^{i\phi}\cos{\theta}\\
\cos{\theta}&-\sin{\theta}\\
\end{pmatrix}
$$


In [13]:
cir = dq.QumodeCircuit(nmode=3, init_state=[1,0,1])
cir.ps(wires=0, inputs=0) # 相位角为0， 作用第0个mode上
cir.ps(1, 0) # 也可省略参数名
cir.bs_theta([0,1], torch.pi/4) # 相位角为pi/4 作用在第0，1个mode上
cir.h(wires=[1,2])  # H gate
cir.bs_rx(wires=[1,2])  # Rx
cir.bs_ry(wires=[0,1])  # Ry
cir.dc(wires=[1,2]) # 50:50 分束器
cir.mzi(wires=[0,1])


通过调用 ``draw()`` 函数实现线路可视化：

In [14]:
cir.draw()

通过调用 ``get_unitary()`` 函数获取线路酉矩阵：

In [15]:
cir.get_unitary()

tensor([[ 0.1470-0.4393j, -0.2474+0.2236j,  0.6414-0.5126j],
        [-0.1533+0.5708j, -0.2599+0.7499j,  0.1236+0.0740j],
        [ 0.3774+0.5419j, -0.3412-0.3774j, -0.1419-0.5338j]],
       grad_fn=<MmBackward0>)

### 前向演化

调用前向函数对线路进行演化。可以通过改变 ``is_prob`` 的值以控制返回的类型：

``is_prob=None`` （默认）：返回量子线路对应的酉矩阵

``is_prob=True`` ：返回末态的概率分布

``is_prob=False`` ：返回末态的振幅分布



In [16]:
print(cir())
print(cir(is_prob=True))
print(cir(is_prob=False))

tensor([[ 0.1470-0.4393j, -0.2474+0.2236j,  0.6414-0.5126j],
        [-0.1533+0.5708j, -0.2599+0.7499j,  0.1236+0.0740j],
        [ 0.3774+0.5419j, -0.3412-0.3774j, -0.1419-0.5338j]],
       grad_fn=<MmBackward0>)
{|200>: tensor([0.2894], grad_fn=<SelectBackward0>), |002>: tensor([0.2661], grad_fn=<SelectBackward0>), |110>: tensor([0.2210], grad_fn=<SelectBackward0>), |011>: tensor([0.1200], grad_fn=<SelectBackward0>), |101>: tensor([0.0890], grad_fn=<SelectBackward0>), |020>: tensor([0.0145], grad_fn=<SelectBackward0>)}
{|200>: tensor([-0.1851-0.5051j], grad_fn=<SelectBackward0>), |002>: tensor([0.3333-0.3937j], grad_fn=<SelectBackward0>), |110>: tensor([0.2450+0.4012j], grad_fn=<SelectBackward0>), |011>: tensor([0.3330+0.0957j], grad_fn=<SelectBackward0>), |101>: tensor([0.2645+0.1380j], grad_fn=<SelectBackward0>), |020>: tensor([-0.0866+0.0837j], grad_fn=<SelectBackward0>)}


支持通过 ``data`` 输入线路中 ``encode=True`` 门的参数，通过 ``state`` 指定初态，对线路进行演化：

In [17]:
cir_2 = dq.QumodeCircuit(2, init_state=[1,0,1])
cir_2.ps(wires=0, encode=True) # 相位角为参数， 作用第0个mode上
cir_2.ps(1, encode =True) # 也可省略参数名
cir_2.bs_theta([0,1], encode=True) # theta为参数, phi为定值pi/2 作用在第0，1个mode上

# 构建data作为输入参数
npara = 3
data = torch.randn(npara)
print('线路参数为:', data)

# 完成前向
cir_2(data)
cir_2.draw()

线路参数为: tensor([-0.7499, -0.0803, -1.1638])


In [18]:
# 指定初态前向演化
print(cir_2(state=[0, 2, 0]))

# 同时指定线路参数和初态前向演化，并返回概率分布
cir_2(data, state=[1, 1, 1], is_prob=True)

tensor([[ 0.2896-0.2698j, -0.0737-0.9154j],
        [-0.6259-0.6720j,  0.3945-0.0317j]])


{|11>: tensor([0.4715]), |02>: tensor([0.2642]), |20>: tensor([0.2642])}

### 采样测量

可以对线路进行测量，返回的结果是字典或者字典的列表，字典的key是fock态，value是对应测量到的次数，shots默认为1024。

In [19]:
cir.measure()

{|011>: 128, |110>: 216, |002>: 273, |200>: 303, |101>: 87, |020>: 17}

设定 ``with_prob=True`` 同时返回理论期望值：

In [20]:
cir.measure(with_prob=True)

{|002>: (261, tensor(0.2661, grad_fn=<AddBackward0>)),
 |200>: (308, tensor(0.2894, grad_fn=<AddBackward0>)),
 |110>: (225, tensor(0.2210, grad_fn=<AddBackward0>)),
 |101>: (87, tensor(0.0890, grad_fn=<AddBackward0>)),
 |020>: (6, tensor(0.0145, grad_fn=<AddBackward0>)),
 |011>: (137, tensor(0.1200, grad_fn=<AddBackward0>))}

通过设定 ``wires`` 进行部分测量：

In [21]:
cir.measure(wires=[0,1])

{|20>: 314, |00>: 259, |01>: 115, |10>: 93, |11>: 228, |02>: 15}

如需要对量子线路进行马尔科夫链蒙特卡洛采样，打开 ``mcmc=True`` 开关即可。

通常只在前向返回酉矩阵，即默认 ``is_prob=None`` 的情况下使用MCMC采样，因为其他情况做前向时已经计算了全空间理论的概率分布。

In [22]:
cir()
cir.measure(mcmc=True)

chain 5: 100%|████████████████████████████████████████| 207/207 [00:00<?, ?it/s]


{|020>: 13, |011>: 125, |110>: 212, |002>: 270, |200>: 315, |101>: 89}

通过 ``get_amplitude()`` 或 ``get_prob()`` 获得指定末态的振幅/概率：

In [23]:
print(cir.get_amplitude([0,1,1]))
print(cir.get_prob([0,1,1]))

tensor([0.3330+0.0957j], grad_fn=<DivBackward0>)
tensor([0.1200], grad_fn=<PowBackward0>)


## 支持batch输入

### 初态的batch输入

In [24]:
# 构建batch的初态
batch_state = torch.randint(0, 3, (4, 2))
print('初态为: ',batch_state)

cir = dq.QumodeCircuit(nmode=2, init_state=batch_state)
cir.bs_theta([0, 1])

re1 = cir()
print('线路对应酉矩阵为: ', re1)

# 测量并输出结果
cir.measure()

初态为:  tensor([[0, 2],
        [0, 0],
        [0, 1],
        [0, 2]])
线路对应酉矩阵为:  tensor([[[-5.6105e-01+0.0000j,  3.6183e-08+0.8278j],
         [-3.6183e-08+0.8278j, -5.6105e-01+0.0000j]],

        [[-5.6105e-01+0.0000j,  3.6183e-08+0.8278j],
         [-3.6183e-08+0.8278j, -5.6105e-01+0.0000j]],

        [[-5.6105e-01+0.0000j,  3.6183e-08+0.8278j],
         [-3.6183e-08+0.8278j, -5.6105e-01+0.0000j]],

        [[-5.6105e-01+0.0000j,  3.6183e-08+0.8278j],
         [-3.6183e-08+0.8278j, -5.6105e-01+0.0000j]]],
       grad_fn=<ExpandBackward0>)


[{|02>: 105, |20>: 496, |11>: 423},
 {|00>: 1024},
 {|10>: 697, |01>: 327},
 {|20>: 495, |11>: 436, |02>: 93}]

### 线路参数的batch输入

In [25]:
# 构建batch的线路参数
batch_size = 10
npara = 1
batch_para = torch.randn(batch_size,npara)
print('线路参数为: ',batch_para)

cir = dq.QumodeCircuit(nmode=2, init_state=[0,1])
cir.bs_theta([0, 1], encode=True)

re1 = cir(data = batch_para)
print('线路对应酉矩阵为: ', re1)

# 测量并输出结果
cir.measure()

线路参数为:  tensor([[ 0.0323],
        [ 1.0679],
        [ 0.8412],
        [ 0.5248],
        [ 0.3026],
        [-0.5640],
        [-0.2673],
        [-0.9447],
        [ 0.8610],
        [-0.8015]])
线路对应酉矩阵为:  tensor([[[ 9.9948e-01+0.0000j,  1.4134e-09+0.0323j],
         [-1.4134e-09+0.0323j,  9.9948e-01+0.0000j]],

        [[ 4.8193e-01+0.0000j,  3.8300e-08+0.8762j],
         [-3.8300e-08+0.8762j,  4.8193e-01+0.0000j]],

        [[ 6.6660e-01+0.0000j,  3.2583e-08+0.7454j],
         [-3.2583e-08+0.7454j,  6.6660e-01+0.0000j]],

        [[ 8.6541e-01+0.0000j,  2.1902e-08+0.5011j],
         [-2.1902e-08+0.5011j,  8.6541e-01+0.0000j]],

        [[ 9.5457e-01+0.0000j,  1.3026e-08+0.2980j],
         [-1.3026e-08+0.2980j,  9.5457e-01+0.0000j]],

        [[ 8.4510e-01+0.0000j, -2.3368e-08-0.5346j],
         [ 2.3368e-08-0.5346j,  8.4510e-01+0.0000j]],

        [[ 9.6448e-01+0.0000j, -1.1546e-08-0.2641j],
         [ 1.1546e-08-0.2641j,  9.6448e-01+0.0000j]],

        [[ 5.8600e-01+0.0000j, -3.

[{|01>: 1021, |10>: 3},
 {|10>: 743, |01>: 281},
 {|01>: 469, |10>: 555},
 {|01>: 749, |10>: 275},
 {|01>: 918, |10>: 106},
 {|01>: 731, |10>: 293},
 {|01>: 951, |10>: 73},
 {|10>: 648, |01>: 376},
 {|10>: 589, |01>: 435},
 {|01>: 483, |10>: 541}]

注意： 当初态和线路参数都为batch输入时，线路会认为每组参数是一对一的，例如初态的第一组参数对应线路的第一组参数，因而batch数需要一致。

In [26]:
# 构建batch初态和线路参数
batch_size = 10
nmode = 2
npara = 1
batch_state = torch.randint(0, 2, (batch_size, nmode))
batch_para = torch.randn(batch_size,npara)
print('batch数为: ', batch_size)
print('初态shape为: ',batch_state.shape)
print('线路参数shape为: ',batch_para.shape)


cir = dq.QumodeCircuit(nmode=nmode, init_state=batch_state)
cir.bs_theta([0, 1], encode=True)

re1 = cir(data = batch_para)
print('线路对应酉矩阵为: ', re1)

# 测量并输出结果
cir.measure()

batch数为:  10
初态shape为:  torch.Size([10, 2])
线路参数shape为:  torch.Size([10, 1])
线路对应酉矩阵为:  tensor([[[-2.6926e-03+0.0000j,  4.3711e-08+1.0000j],
         [-4.3711e-08+1.0000j, -2.6926e-03+0.0000j]],

        [[ 9.0478e-01+0.0000j,  1.8615e-08+0.4259j],
         [-1.8615e-08+0.4259j,  9.0478e-01+0.0000j]],

        [[ 5.1700e-01+0.0000j, -3.7416e-08-0.8560j],
         [ 3.7416e-08-0.8560j,  5.1700e-01+0.0000j]],

        [[ 9.4056e-01+0.0000j, -1.4846e-08-0.3396j],
         [ 1.4846e-08-0.3396j,  9.4056e-01+0.0000j]],

        [[ 3.7369e-01+0.0000j,  4.0545e-08+0.9276j],
         [-4.0545e-08+0.9276j,  3.7369e-01+0.0000j]],

        [[ 9.9559e-01+0.0000j,  4.0998e-09+0.0938j],
         [-4.0998e-09+0.0938j,  9.9559e-01+0.0000j]],

        [[ 5.9964e-01+0.0000j, -3.4981e-08-0.8003j],
         [ 3.4981e-08-0.8003j,  5.9964e-01+0.0000j]],

        [[ 9.9944e-01+0.0000j, -1.4684e-09-0.0336j],
         [ 1.4684e-09-0.0336j,  9.9944e-01+0.0000j]],

        [[ 3.2596e-01+0.0000j,  4.1324e-08+0.945

[{|00>: 1024},
 {|00>: 1024},
 {|10>: 723, |01>: 301},
 {|01>: 887, |10>: 137},
 {|01>: 888, |10>: 136},
 {|11>: 989, |02>: 23, |20>: 12},
 {|00>: 1024},
 {|10>: 1022, |01>: 2},
 {|10>: 940, |01>: 84},
 {|10>: 953, |01>: 71}]

## 支持简单的噪声模拟（Gaussian noise）

噪声模拟使用 ``QumodeCircuit(noise=True)`` , 这里默认所有参数的噪声都是高斯分布，平均值 ``mu`` 为0, 标准差 ``sigma`` 为0.1。支持自定义整条线路的全局噪声，也可以指定某些光学器件的噪声。

In [27]:
cir_n = dq.QumodeCircuit(nmode=3, init_state=[1,0,1], noise=True)
cir_n.ps(wires=0, encode=True, mu=0.2, sigma=0.2) # 支持自定义mu和sigma的值，默认为0，0.1
cir_n.ps(1, encode=True, mu=0, sigma=0)
cir_n.bs_theta([0,1], encode=True, mu=0, sigma=0.3)

# 定义线路参数，并前向演化保存末态
para = torch.randn(3)
final_state_n = cir_n(para, is_prob=False)

print('采样结果为：', cir_n.measure(with_prob=True))

采样结果为： {|101>: (1024, tensor(0.9999))}


与无噪声的情况进行比较，计算fidelity：

In [28]:
# 设置无噪声线路
cir = dq.QumodeCircuit(nmode=3, init_state=[1,0,1])
cir.ps(0, encode=True)
cir.ps(1, encode=True)
cir.bs_theta([0,1], encode=True)

final_state = cir(para, is_prob=False)

print('采样结果为：', cir.measure(with_prob=True))

# 计算fidelity
f = 0
for key in final_state_n.keys():
    f += final_state_n[key].conj()*final_state[key]
fidelity = torch.abs(f)**2
print('加入噪声后的fidelity为: ', fidelity)

采样结果为： {|101>: (996, tensor(0.9727)), |011>: (28, tensor(0.0273))}
加入噪声后的fidelity为:  tensor([0.9703])


## 支持GPU计算



由于DeepQuantum基于PyTorch框架，可以方便的进行GPU节点上的量子线路模拟。

首先检查环境是否支持cuda：

In [29]:
torch.cuda.is_available()

True

如果有NVIDIA的GPU但仍然返回 ``False`` ，常见的原因是未安装或未安装合适版本的 [CUDA](https://anaconda.org/nvidia/cuda) ，或安装的PyTorch为CPU-only版本。

In [30]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

cir = dq.QumodeCircuit(nmode=nmode, init_state=[2,0])
cir.bs_theta([0, 1], encode=True)

cir.to(device)
batch_para = batch_para.to(device)
print(batch_para)

re1 = cir(data = batch_para)
print('线路对应酉矩阵为: ', re1)

# 测量并输出结果
cir.measure()

tensor([[ 1.5735],
        [ 0.4399],
        [-1.0275],
        [-0.3465],
        [ 1.1878],
        [ 0.0939],
        [-0.9277],
        [-0.0336],
        [ 1.2388],
        [ 1.8277]], device='cuda:0')
线路对应酉矩阵为:  tensor([[[-2.6926e-03+0.0000j,  4.3711e-08+1.0000j],
         [-4.3711e-08+1.0000j, -2.6926e-03+0.0000j]],

        [[ 9.0478e-01+0.0000j,  1.8615e-08+0.4259j],
         [-1.8615e-08+0.4259j,  9.0478e-01+0.0000j]],

        [[ 5.1700e-01+0.0000j, -3.7416e-08-0.8560j],
         [ 3.7416e-08-0.8560j,  5.1700e-01+0.0000j]],

        [[ 9.4056e-01+0.0000j, -1.4846e-08-0.3396j],
         [ 1.4846e-08-0.3396j,  9.4056e-01+0.0000j]],

        [[ 3.7369e-01+0.0000j,  4.0545e-08+0.9276j],
         [-4.0545e-08+0.9276j,  3.7369e-01+0.0000j]],

        [[ 9.9559e-01+0.0000j,  4.0998e-09+0.0938j],
         [-4.0998e-09+0.0938j,  9.9559e-01+0.0000j]],

        [[ 5.9964e-01+0.0000j, -3.4981e-08-0.8003j],
         [ 3.4981e-08-0.8003j,  5.9964e-01+0.0000j]],

        [[ 9.9944e-01+0.0

[{|02>: 1024},
 {|20>: 686, |02>: 32, |11>: 306},
 {|11>: 407, |02>: 548, |20>: 69},
 {|11>: 197, |20>: 816, |02>: 11},
 {|20>: 28, |02>: 751, |11>: 245},
 {|20>: 1014, |11>: 10},
 {|02>: 412, |11>: 493, |20>: 119},
 {|20>: 1021, |11>: 3},
 {|02>: 831, |11>: 184, |20>: 9},
 {|02>: 881, |11>: 138, |20>: 5}]

## Clements 架构光量子线路

内置了Clements模块可以模拟任意的酉矩阵，通过`dq.Clements`直接调用, 
`nmode` 表示总的波导数目，`init_state`设置初始输入，`cutoff` 表示对输出的结果做截断，`phi_first` 表示不同MZI的构造， `phi_first=True` 表示PS-BS-PS-BS 结构， `noise` 表示是否引入高斯噪声。
下面是使用Clements 模块构建相同的CNOT光量子线路的例子。

1. 构建6模Clements线路， 这里没有指定参数，所以生成的是随机参数。

In [31]:
clements = dq.Clements(nmode=6, init_state=[1,0,1,0,0,0], cutoff=3)
clements.draw()

2. 使用clements架构实现cnot门, 这里先将CNOT对应的光量子线路酉矩阵做分解映射成Clements参数， 然后再加载这些参数。

In [32]:
# 使用clements架构实现cnot门
u6x6 = np.array([[1, 0, 1, -1, 0, 0],
                 [0, 1, 0 ,0,  0, np.sqrt(2)],
                 [1,  0, 0, 1, 1, 0],
                 [-1, 0, 1, 0, 1, 0],
                 [0,  0, 1, 1, -1,0],
                 [0, np.sqrt(2), 0,0,0,-1]])/np.sqrt(3)
# 将酉矩阵分解成clements对应的参数
UD = dq.UnitaryDecomposer(u6x6)
mzi_info = UD.decomp()
# 构造clements线路实现cnot门
data = clements.dict2data(mzi_info[2]) # encoding the 6x6 data
re = clements(data = data)
#线路可视化
clements.draw()

3. 通过clements.get_unitary()来验证是否正确

In [33]:
abs(clements.get_unitary() - u6x6).sum()

tensor(0.0007, dtype=torch.float64)

# 基于高斯后端构建光量子线路

##  构建连续变量光量子线路

基于高斯后端的光量子线路演化的量子态是高斯态， 高斯态的概念来源于连续变量光量子计算， 它对应的wigner函数为多元高斯分布，比如相干态。
这里用正交分量X，P的协方差矩阵和平均值来刻画连续变量中的量子态，比如真空态对应的协方差矩阵为单位阵，对应的平均值都为0，
通过测量得到的物理量是两个正交分量X，P的值，它们对应的分布满足边缘分布。


这里构建2模线路，初始量子态为真空高斯态， 经过3种高斯门演化之后得到另一个高斯态。S 表示单模真空压缩门， D表示位移门， BS是可调分束器。 通过设置`backend='gaussian'`可以进行高斯后端的演化得到协方差矩阵和平均值

In [34]:
circ = dq.photonic.QumodeCircuit(nmode=2, init_state='vac', cutoff=3, backend='gaussian')
circ.s(wires=[0], r=1)
circ.s(wires=[1], r=1)
circ.d(wires=[0], r=1)
circ.d(wires=[1], r=1)
circ.bs(wires=[0,1], inputs=[np.pi/4,np.pi/4])
#线路可视化
circ.draw()

In [35]:
state = circ() # 线路演化后得到的高斯态用协方差矩阵和平均值刻画
print(state)

[tensor([[[ 1.9488,  0.0000,  1.8134, -2.5646],
         [ 0.0000,  1.9488, -2.5646, -1.8134],
         [ 1.8134, -2.5646,  5.5756,  0.0000],
         [-2.5646, -1.8134,  0.0000,  5.5756]]]), tensor([[[0.4142],
         [2.4142],
         [1.0000],
         [1.0000]]])]


## homodyne 测量

`circ.measure_homodyne` 对指定线路的正交分量进行测量，`shots`对应测量次数， `wires` 表示指定线路

In [36]:
measure_re = circ.measure_homodyne(shots = 1024, wires=[0,1]) #对正交分量进行测量， 这里的每个模对应一组正交分量X，P。
measure_re.shape

torch.Size([1024, 4])

同时还可以计算每个线路的平均光子数和光子数方差

In [37]:
photon_mean, photon_var = circ.photon_number_mean_var()
photon_mean, photon_var

(tensor([1.6740, 3.0882]), tensor([6.7859, 6.9773]))

## 高斯玻色采样

基于压缩门和线性光学器件可以构建高斯玻色采样线路，参数 `adj_mat` 表示输入对应的邻接矩阵，`cutoff`表示每条线路探测到的最大光子数-1，`mean_photon_num` 表示每条线路对应的平均光子数之和， 默认设为mode数，即每条线路对应的平均光子数为1， `detector` 为
`'pnrd'`表示单光子分辨探测器， 默认设为`'pnrd', `'threshold'` 表示阈值探测器， `noise` 表示是否加入高斯噪声。

In [38]:
# 采用6个节点的图演示简单的高斯玻色采样
a = np.array([[0., 1., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0.]])
gbs = dq.photonic.ansatz.GBS_Graph(adj_mat=torch.tensor(a), cutoff=2)
state = gbs()
sample = gbs.measure(shots=5000)
gbs.draw()

chain 5: 100%|█████████████████████████████| 999/999 [00:00<00:00, 20173.86it/s]


In [39]:
sample

{|011101>: 1,
 |000110>: 359,
 |000101>: 467,
 |110000>: 337,
 |000000>: 2311,
 |010001>: 267,
 |010100>: 411,
 |101000>: 393,
 |101110>: 52,
 |110101>: 57,
 |111100>: 63,
 |010111>: 61,
 |101101>: 88,
 |110110>: 63,
 |111001>: 63,
 |111111>: 3,
 |011011>: 1,
 |001111>: 1,
 |001010>: 2}

## 部分homodyne测量

高斯后端还可以对几个mode做部分测量， 那么剩余的量子态会根据相应的测量结果塌缩, 通过`circ.homodyne` 实现，`wires`表示作用单条线路，`phi`对应测量角度。

In [40]:
circ = dq.photonic.QumodeCircuit(nmode=2, init_state='vac', cutoff=3, backend='gaussian')
circ.s(wires=[0], r=1)
circ.s(wires=[1], r=1)
circ.d(wires=[0], r=1)
circ.d(wires=[1], r=1)
circ.bs(wires=[0,1], inputs=[np.pi/4,np.pi/4])
circ.homodyne(wires = 0, phi=0)
#线路可视化
circ.draw()

In [41]:
state = circ()
sample = circ.measure_homodyne(shots=1) # 运行线路并进行一次采样
state_measured = circ.state_measured # 得到采样后坍缩的量子态

In [42]:
sample, state_measured

(tensor(1.5715),
 [tensor([[[ 1.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  1.9488,  0.0000, -1.8134],
           [ 0.0000,  0.0000,  1.0000,  0.0000],
           [ 0.0000, -1.8134,  0.0000,  2.2006]]]),
  tensor([[[ 0.0000],
           [ 2.4146],
           [ 0.0000],
           [-0.5233]]])])

## 延时线圈组成的时分复用线路

通过`cir.delay`加入可调延时线圈，参数`wires` 表示作用的波导，`ntau`表示延时线圈内延时的个数， `inputs` 表示延时线圈中可调参数， `convention` 对应两种，`bs`和`mzi`, 默认是`bs`, 对应延时线有一个可调BS和PS. `encode` 控制角度自由编码。

In [43]:
r=5
nmode = 2
cir = dq.QumodeCircuitTDM(nmode=nmode, init_state='vac', cutoff=3)
cir.s(0)
cir.s(1)
cir.bs_theta([0,1], [3])
cir.delay(0, ntau = 1, inputs=[3,0], encode=True)
cir.delay(1, ntau = 1, inputs=[3,0], encode=True)
cir.draw()

通过控制`encode=True`可以实现对不同时刻编码不同角度，从而制备不同的纠缠态，下面是EPR态的例子, 第一次测量设置延时线圈参数为
$[\pi/2, \pi/2]$, 第二次测量设置为$[\pi/4, 0]$， 然后周期性重复角度测量可以得到纠缠的EPR态。

In [44]:
r=9
nmode = 1
cir = dq.QumodeCircuitTDM(nmode=nmode, init_state='vac', cutoff=3)
cir.s(0, r=r)
cir.delay(0, ntau=1, inputs=[np.pi/2, np.pi/2], encode=True) # 参数编码
cir.homodyne_x(0)
# cir.draw()
data1 = torch.tensor([[np.pi/2, np.pi/2],
                      [np.pi/4, 0]]) # 周期性参数组合
data2 = data1.unsqueeze(0)
test = cir(data=data2, nstep=13)
sample = cir.samples
sample

tensor([ 3.5459e-01,  1.9252e+03,  1.9252e+03, -5.3627e+03, -5.3627e+03,
         3.9202e+03,  3.9202e+03,  1.6300e+03,  1.6300e+03, -2.3322e+03,
        -2.3322e+03, -5.9210e+03, -5.9210e+03])

### 空间线路的转译

通过`cir.draw(unroll=True)` 可以得到初次测量的等效的空间线路 

In [45]:
cir.draw(unroll=True)

若要进行n次测量实验， 得到对应的空间等效线路，可以通过`circ.global_circuit` , 设置`nstep`得到

In [46]:
cir_global = cir.global_circuit(nstep=3)
cir_global.draw(unroll=True) # draw global circuit

当设置对应的`encode=True`, 就可以自由编码global 线路的角度

In [47]:
data = torch.tensor([0]*cir_global.ndata)
state = cir_global(data=data)

In [48]:
cir_global.draw(unroll=True) # draw global circuit